In [1]:
import numpy as np
import pandas as pd
import os
import sqlite3
from nltk.corpus import stopwords
sw = stopwords.words('english')
sw2 = stopwords.words("english")
from string import punctuation
punctuation = set(punctuation)
punctuation.add("’")
import datetime
import time
from random import sample
import random
from IPython.display import Image

import tweepy
from tweepy import OAuthHandler
# I've put my API keys in a .py file called API_keys.py
from my_api_keys import api_key, api_key_secret, access_token, access_token_secret

import json
import csv
import re
import string
from collections import Counter, defaultdict
from pprint import pprint
from operator import itemgetter
import matplotlib.pyplot as plt

import pyLDAvis
import pyLDAvis.gensim_models
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel,LdaMulticore, Phrases 
from gensim.models.phrases import Phraser 
from gensim.corpora import Dictionary
nlp = spacy.load('en_core_web_sm')
Lemmatizer = nlp.get_pipe("lemmatizer")

import nltk
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from matplotlib.pyplot import text

/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
# Modifications to punctuation and sw lists

punctuation = set(punctuation)
punctuation.add("’")

sw2 = set(sw)
addl = ("|","-","/","•","&", "&amp;")
sw2.update(addl)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


####  Read daily tweets CSVs into pandas dataframe

In [3]:
db = pd.DataFrame(columns = ['user_id','screen_name','description','location','friends_count',
           'followers_count','totaltweets','date_created', 'tweet_id', 'retweetcount','full_text'])

file_location = "/Users/natebender/Desktop/Repo/footprints/footprints_audience/data/"
files = sorted(os.listdir(file_location))
for idx, file in enumerate(files):
    
    data = "".join([file_location,file])
    datafile = pd.read_csv(data)

    db = db.append(datafile,ignore_index=True)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# Check desc stats on overall descriptions before splitting into groups
def get_patterns(all_tweets) :

    all_desc = all_tweets.dropna()
    all_str = " ".join(all_desc)    
    clean = [w for w in all_str.split() if w.lower() not in sw2]
    
    # Calculate your statistics here
    total_tokens = len(clean)
    unique_tokens = len(set(clean))
    clean_tok_len = [len(w) for w in clean]
    avg_token_len = np.mean(clean_tok_len)
    lex_diversity = len(set(clean))/len(clean)
    top_n = Counter(clean).most_common(20)
    
    
    # Now we'll fill out the dictionary. 
    results = {'tokens':total_tokens,
               'unique_tokens':unique_tokens,
               'avg_token_length':round(avg_token_len,2),
               'lexical_diversity':round(lex_diversity,2),
               'Top_n':top_n}

    return(results)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
all_tweets = db.full_text.dropna()
print(f'Database: {"{:,}".format(len(db.tweet_id))} tweets')
print(f'Descriptive stats are:')
get_patterns(all_tweets)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Database: 73,354 tweets
Descriptive stats are:


{'tokens': 1366310,
 'unique_tokens': 64709,
 'avg_token_length': 7.87,
 'lexical_diversity': 0.05,
 'Top_n': [('#running', 46883),
  ('#run', 15784),
  ('run', 11013),
  ('#fitness', 7290),
  ('running', 5917),
  ('#Running', 5637),
  ('#trailrunning', 4823),
  ('#runner', 4416),
  ('miles', 3869),
  ('get', 3721),
  ('#ukrunchat', 3708),
  ('.', 3651),
  ('day', 3475),
  ('2022', 3158),
  ('new', 2961),
  ('year', 2928),
  ('Running', 2900),
  ('morning', 2856),
  ('Run', 2825),
  ('#workout', 2774)]}

### Topic Modeling

Now we want to dig in deeper and see if we can find groups of tweets that cluster together by distinct theme. 

We'll accomplish this using LDA (latent dirichlet analysis) modeling. LDA is an unsupervised classification algorithm.

In [6]:
for_modeling_unclean = db.full_text.dropna()
for_modeling = []

for tweet in for_modeling_unclean :
    words = [w for w in tweet.split()]# if w not in cop_sw]
    words = " ".join(words)    
    for_modeling.append(words)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
len(for_modeling)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


73354

In [8]:
# database is small enough right now (1/13/22) that taking a sample is not necessary
# random.seed(1234)
# for_modeling = random.sample(for_modeling, 50000)  

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
# Updates spaCy's default stop words list with my additional words. 
# stop_list = ['`',"Mr.","Mrs.","Ms."]
# nlp.Defaults.stop_words.update(stop_list)

# Iterates over the words in the stop words list and resets the "is_stop" flag.
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
program_start = time.time()

doc_list = []
allowed_postags=['NOUN','ADJ','VERB','ADV']

# Iterates through each article in the corpus.
for doc in for_modeling :
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(doc)
    doc_list.append([token.lemma_ for token in pr if token.pos_ in allowed_postags])
    
program_end = time.time()
print('Total time taken to run is {} minutes.'.format(round(program_end - program_start)/60, 2))

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Total time taken to run is 30.816666666666666 minutes.


In [11]:
id2word = Dictionary(doc_list)  
id2word.filter_extremes(no_below=10, no_above=0.4)  #getting rid of fewer than 10 instances. 
                                # And no more than words that appear in a certain fraction
                                # of the total corpus size (in this case .4)
id2word.compactify()  # assign new word ids to all words. 
corpus = [id2word.doc2bow(word) for word in doc_list]


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
num_topics = 5

program_start = time.time()

lda_model = LdaMulticore(corpus=corpus, 
                             id2word=id2word, 
                             num_topics=num_topics, 
                             random_state=1,
                             chunksize=30,
                             passes=20,
                             alpha=0.31,  # sets our priors
                             eta=0.91,
                             eval_every=1,
                             per_word_topics=True,
                             workers=1)

program_end = time.time()
print('Total time taken to run is {} minutes.'.format(round(program_end - program_start)/60, 2))

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Total time taken to run is 13.166666666666666 minutes.


### LDA Modeling

Through trial and error, I found that asking the LDA model for 10 groups gave me the most valuable breakdown of groups overall, even though a few of the groups themselves aren't that useful. For instance, when restricting the group size down to seven or eight, the Education-focused group gets folded into one of the other groups while another group of fairly random event-focused verbs appears. By expanding the group size to 10 I get more useful distinctions and can get ignore the less-useful groups when I go to interpret the group sentiment. 

In [13]:
pprint(lda_model.print_topics(num_words=10))

[(0,
  '0.023*"day" + 0.023*"today" + 0.021*"mile" + 0.017*"go" + 0.016*"do" + '
  '0.016*"morning" + 0.014*"get" + 0.014*"so" + 0.013*"time" + 0.011*"good"'),
 (1,
  '0.028*"trail" + 0.026*"trailrunne" + 0.023*"shoe" + 0.015*"nature" + '
  '0.013*"beautiful" + 0.012*"new" + 0.010*"hike" + 0.008*"photo" + '
  '0.008*"check" + 0.008*"love"'),
 (2,
  '0.017*"marathon" + 0.013*"runner" + 0.013*"more" + 0.012*"race" + '
  '0.009*"running" + 0.009*"here" + 0.008*"life" + 0.008*"year" + 0.008*"get" '
  '+ 0.008*"have"'),
 (3,
  '0.056*"fitness" + 0.029*"workout" + 0.021*"exercise" + 0.016*"training" + '
  '0.016*"health" + 0.014*"gym" + 0.014*"sport" + 0.011*"cycle" + '
  '0.010*"fitnessmotivation" + 0.010*"free"'),
 (4,
  '0.049*"️" + 0.037*"running" + 0.028*"capetownrunne" + 0.027*"session" + '
  '0.023*"bootcamp" + 0.022*"♂" + 0.021*"day" + 0.021*"capetownretreat" + '
  '0.021*"begin" + 0.020*"capetownt"')]


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [56]:
pyLDAvis.enable_notebook()

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [57]:
#pyLDAvis.gensim.prepare(lda_model, corpus, words)
#pyLDAvis.gensim_models.prepare(lda_model, corpus,id2word)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
# create temp lists to pass into dataframe
tweetlist = []
categorylist = []
probabilitylist = []
topic_assignments = []


# Calculate topic probabilities for each tweet & assign tweets to categories
for tweet in for_modeling :
    doc = [w for w in tweet.split()]
    pr = nlp(" ".join(doc))
    doc = [token.lemma_ for token in pr if token.pos_ in allowed_postags]
    doc_new = id2word.doc2bow(doc)

    topic_probs = lda_model[doc_new][0]
    topic = max(topic_probs,key=lambda x: x[1])
    topic_assignments.append(topic[0])    
    prob = max(topic_probs)
    cat = topic[0]
    prob = topic[1]
    tweetlist.append(tweet)
    categorylist.append(cat)
    probabilitylist.append(prob)

tweets_df = pd.DataFrame()
tweets_df["Tweet"] = tweetlist
tweets_df["Category"] = categorylist
tweets_df["Probability"] = probabilitylist

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
tweets_df

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Tweet,Category,Probability
0,Happy New Year! #run https://t.co/MzfmqWLfSm,0,0.200000
1,Hard packed snow trail run = good workout! Fre...,1,0.465835
2,🚨 #AfterChristmasSale 🚨 ✨Last Day!✨ 40%OFF #pi...,1,0.699859
3,It is so beautiful today in #sanfrancisco so I...,0,0.753919
4,Breathing correctly when you #run is important...,2,0.600812
...,...,...,...
73349,5K am #run in PDX and another pm 5K with a vie...,0,0.729507
73350,Out for a little run. Whitehorse Yukon 🪶🪶🪶🪶 #B...,0,0.644757
73351,(RRW) Athletics: Race Results Weekly Among 202...,3,0.640317
73352,1.61 @CharityMiles 4 @nothingbutnets. Thx @Asp...,3,0.898965


In [16]:
newdf = tweets_df.merge(db, left_on='Tweet', right_on='full_text', how='inner')
newdf = newdf.drop_duplicates(subset=['tweet_id'], keep='first')
newdf["date_created"] = pd.to_datetime(newdf["date_created"], format="%Y-%m-%d") 
newdf['days'] = newdf['date_created'].dt.date

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
# # rename columns from numbers to actual names for final display

# newdf['Category'] = newdf['Category'].replace([0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
#                                                             ['Random','Activism','Event-focused random','Collaboration',
#                                                              'Fossil Fuels', 'Education','CTAs & tracking',
#                                                              'End Bad Policy','Aspirational Policy','National FF Policy'])
                                                             


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
newdf_groupby_category = newdf.groupby('Category')

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Why are we getting so few tweets in the categories if every tweet is being categorized into a category? Across the eight categories we should see the same total as number of tweets in the database?

In [19]:
print(f"Number of tweets per category")
print(newdf_groupby_category.size())

Number of tweets per category
Category
0    2765
1     804
2    1058
3     465
4     351
dtype: int64


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Topic modeling the Environmental group

In [20]:
newdf_env = newdf[newdf['Category']==1]

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
newdf_env_unique = newdf_env.drop_duplicates(subset = ["user_id"])
print(len(newdf_env_unique))

447


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
for_modeling_unclean = newdf_env_unique.full_text.dropna()
for_modeling = []

for tweet in for_modeling_unclean :
    words = [w for w in tweet.split()]# if w not in cop_sw]
    words = " ".join(words)    
    for_modeling.append(words)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
program_start = time.time()

doc_list = []
allowed_postags=['NOUN','ADJ','VERB','ADV']

# Iterates through each article in the corpus.
for doc in for_modeling :
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(doc)
    doc_list.append([token.lemma_ for token in pr if token.pos_ in allowed_postags])
    
program_end = time.time()
print('Total time taken to run is {} minutes.'.format(round(program_end - program_start)/60, 2))

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Total time taken to run is 0.2833333333333333 minutes.


In [25]:
id2word = Dictionary(doc_list)  
id2word.filter_extremes(no_below=10, no_above=0.4)  #getting rid of fewer than 10 instances. 
                                # And no more than words that appear in a certain fraction
                                # of the total corpus size (in this case .4)
id2word.compactify()  # assign new word ids to all words. 
corpus = [id2word.doc2bow(word) for word in doc_list]


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
num_topics = 3

program_start = time.time()

lda_model = LdaMulticore(corpus=corpus, 
                             id2word=id2word, 
                             num_topics=num_topics, 
                             random_state=1,
                             chunksize=30,
                             passes=40,
                             alpha=0.31,  # sets our priors
                             eta=0.91,
                             eval_every=1,
                             per_word_topics=True,
                             workers=1)

program_end = time.time()
print('Total time taken to run is {} minutes.'.format(round(program_end - program_start)/60, 2))

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Total time taken to run is 0.3333333333333333 minutes.


In [27]:
pprint(lda_model.print_topics(num_words=10))

[(0,
  '0.099*"trailrunne" + 0.065*"trail" + 0.047*"morning" + 0.046*"mile" + '
  '0.044*"beautiful" + 0.039*"new" + 0.035*"winter" + 0.034*"day" + '
  '0.033*"snow" + 0.032*"trailrunning"'),
 (1,
  '0.107*"shoe" + 0.055*"running" + 0.037*"wealth" + 0.034*"sale" + '
  '0.033*"shopping" + 0.032*"style" + 0.031*"fashion" + 0.031*"brand" + '
  '0.031*"nike" + 0.030*"find"'),
 (2,
  '0.116*"trail" + 0.092*"nature" + 0.077*"hike" + 0.075*"walk" + '
  '0.036*"shirt" + 0.033*"outdoors" + 0.033*"hiking" + 0.032*"dog" + '
  '0.032*"travel" + 0.029*"adventure"')]


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
# create temp lists to pass into dataframe
tweetlist = []
categorylist = []
probabilitylist = []
topic_assignments = []


# Calculate topic probabilities for each tweet & assign tweets to categories
for tweet in for_modeling :
    doc = [w for w in tweet.split()]
    pr = nlp(" ".join(doc))
    doc = [token.lemma_ for token in pr if token.pos_ in allowed_postags]
    doc_new = id2word.doc2bow(doc)

    topic_probs = lda_model[doc_new][0]
    topic = max(topic_probs,key=lambda x: x[1])
    topic_assignments.append(topic[0])    
    prob = max(topic_probs)
    cat = topic[0]
    prob = topic[1]
    tweetlist.append(tweet)
    categorylist.append(cat)
    probabilitylist.append(prob)

newdf_env_temp = pd.DataFrame()
newdf_env_temp["Tweet"] = tweetlist
newdf_env_temp["Category"] = categorylist
newdf_env_temp["Probability"] = probabilitylist

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
tweets_df

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Tweet,Category,Probability
0,Happy New Year! #run https://t.co/MzfmqWLfSm,0,0.200000
1,Hard packed snow trail run = good workout! Fre...,1,0.465835
2,🚨 #AfterChristmasSale 🚨 ✨Last Day!✨ 40%OFF #pi...,1,0.699859
3,It is so beautiful today in #sanfrancisco so I...,0,0.753919
4,Breathing correctly when you #run is important...,2,0.600812
...,...,...,...
73349,5K am #run in PDX and another pm 5K with a vie...,0,0.729507
73350,Out for a little run. Whitehorse Yukon 🪶🪶🪶🪶 #B...,0,0.644757
73351,(RRW) Athletics: Race Results Weekly Among 202...,3,0.640317
73352,1.61 @CharityMiles 4 @nothingbutnets. Thx @Asp...,3,0.898965


In [37]:
newdf_env_temp

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Tweet,Category,Probability
0,"On the treadmill 🏃🏻‍♀️🏃🏻‍♀️🏃🏻‍♀️, but still sa...",0,0.787505
1,Rail trail run to Quakertown @jaytee43 @alitvi...,0,0.535058
2,4 Tube Fitness Resistance Bands #running #shoe...,1,0.678286
3,"Pretty warm in #Melbourne already, but got out...",0,0.595775
4,Yoga &amp; meditation. Morning yoga with the g...,2,0.590609
...,...,...,...
799,#running #shoes Low Waist Push Up Leggings htt...,1,0.787324
800,Beautiful sunshine to heal those #running legs...,0,0.558365
801,Women's High Waist Pants #running #wealth http...,1,0.678019
802,#trail #trailwalk #iphonography #shotoniphone ...,2,0.784055


In [39]:
newdf_env_final = newdf_env_temp.merge(tweets_df, left_on='Tweet', right_on='Tweet', how='inner')

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
newdf_env_final

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Tweet,Category_x,Probability_x,Category_y,Probability_y
0,"On the treadmill 🏃🏻‍♀️🏃🏻‍♀️🏃🏻‍♀️, but still sa...",0,0.787505,1,0.416757
1,"On the treadmill 🏃🏻‍♀️🏃🏻‍♀️🏃🏻‍♀️, but still sa...",0,0.787505,1,0.416748
2,"On the treadmill 🏃🏻‍♀️🏃🏻‍♀️🏃🏻‍♀️, but still sa...",0,0.787505,1,0.416759
3,"On the treadmill 🏃🏻‍♀️🏃🏻‍♀️🏃🏻‍♀️, but still sa...",0,0.787505,1,0.416763
4,"On the treadmill 🏃🏻‍♀️🏃🏻‍♀️🏃🏻‍♀️, but still sa...",0,0.787505,1,0.416778
...,...,...,...,...,...
5693,Unisex Waist &amp; Hip Bag with Water Bottle #...,1,0.678286,1,0.513712
5694,Unisex Waist &amp; Hip Bag with Water Bottle #...,1,0.678286,1,0.513712
5695,Unisex Waist &amp; Hip Bag with Water Bottle #...,1,0.678286,1,0.513712
5696,Unisex Waist &amp; Hip Bag with Water Bottle #...,1,0.678286,1,0.513712


In [40]:
newdf_envfinal_cats = newdf_env_final.groupby('Category')

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


KeyError: 'Category'

In [33]:
print(f"Number of tweets per category")
print(newdf_envfinal_cats.size())

Number of tweets per category
Category
1    447
dtype: int64


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
newdf_final = newdf_env_unique[newdf_env_unique['Category']==1]

In [30]:
pyLDAvis.enable_notebook()
# pyLDAvis.gensim.prepare(lda_model, corpus, words)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
